# Create manifests

In [25]:
from os.path import basename
from pathlib import Path
import pandas as pd
import lib.db as db
import lib.util as util

### MOBOT Manifest

In [4]:
CXN = db.connect()
INTERIM_DATA = Path('..') / 'data' / 'interim'

In [17]:
taxonomy = pd.read_sql('SELECT * FROM taxons', CXN)

sql = """SELECT *
           FROM images
          WHERE file_name LIKE '%/MO-DOE-nitfix_specimen_photos/%'"""

images = pd.read_sql(sql, CXN)

taxons = {}
for key, taxon in taxonomy.iterrows():
    guids = util.split_uuids(taxon.sample_ids)
    for guid in guids:
        taxons[guid] = taxon.scientific_name

In [20]:
for key, image in images.iterrows():
    images.loc[key, 'resolved_name'] = taxons.get(image.sample_id)

images.head()

,sample_id,file_name,resolved_name
0,0e6f248b-aa6c-40f1-b571-1f0db94ad3a3,../data/raw/MO-DOE-nitfix_specimen_photos/R000...,Abrus fruticulosus
1,0e669154-9371-441d-85fa-683c1761b184,../data/raw/MO-DOE-nitfix_specimen_photos/R000...,Abrus schimperi
2,0e641ae8-5219-495f-8c72-7b76449c67bc,../data/raw/MO-DOE-nitfix_specimen_photos/R000...,Adenocarpus anagyrifolius
3,0e5e5134-11b2-47ee-aef3-6b1d753b172f,../data/raw/MO-DOE-nitfix_specimen_photos/R000...,Adenocarpus bacquei
4,0e5a4cc5-bd58-4062-8918-94e0c8e9fa7e,../data/raw/MO-DOE-nitfix_specimen_photos/R000...,Adenocarpus decorticans


In [33]:
images.file_name = images.file_name.apply(basename)
print(len(images))
images.head()

1024


,sample_id,file_name,resolved_name
0,0e6f248b-aa6c-40f1-b571-1f0db94ad3a3,R0002644.JPG,Abrus fruticulosus
1,0e669154-9371-441d-85fa-683c1761b184,R0002645.JPG,Abrus schimperi
2,0e641ae8-5219-495f-8c72-7b76449c67bc,R0002646.JPG,Adenocarpus anagyrifolius
3,0e5e5134-11b2-47ee-aef3-6b1d753b172f,R0002647.JPG,Adenocarpus bacquei
4,0e5a4cc5-bd58-4062-8918-94e0c8e9fa7e,R0002648.JPG,Adenocarpus decorticans


In [31]:
csv_path = INTERIM_DATA / 'mobot_manifest.csv'
images.to_csv(csv_path, index=False)

In [32]:
missing = images.resolved_name.isna()
missing_images = images[missing]
len(missing_images)

32